In [1]:
%pwd

'/home/ste/Documents/gesture-gaze-extension/notebooks'

In [5]:
import os
import cv2
import json
from tqdm import tqdm

INPUT_IMG_DIR = "/home/ste/Documents/gesture-gaze-extension/datasets/SynthGaze/lit"
INPUT_JSON_DIR = "/home/ste/Documents/gesture-gaze-extension/datasets/SynthGaze/annotation"
OUTPUT_IMG_DIR = "/home/ste/Documents/gesture-gaze-extension/datasets/SynthGazeProcessed/images"
OUTPUT_JSON_DIR = "/home/ste/Documents/gesture-gaze-extension/datasets/SynthGazeProcessed/json"

TARGET_SIZE = 224
ORIG_WIDTH, ORIG_HEIGHT = 640, 480

In [6]:
CROP_LEFT = (ORIG_WIDTH - ORIG_HEIGHT) // 2  # 80
SCALE = TARGET_SIZE / ORIG_HEIGHT  # 224 / 480

os.makedirs(OUTPUT_IMG_DIR, exist_ok=True)
os.makedirs(OUTPUT_JSON_DIR, exist_ok=True)

def preprocess_image(img):
    cropped_img = img[:, CROP_LEFT:ORIG_WIDTH-CROP_LEFT]
    resized_img = cv2.resize(cropped_img, (TARGET_SIZE, TARGET_SIZE))
    return resized_img

def transform_point(x, y):
    x_cropped = max(0, x - CROP_LEFT)
    return x_cropped * SCALE, y * SCALE

def transform_json(data):
    transformed = {
        "Overall": data["Overall"],
        "Landmarks": {}
    }
    for key, (x, y) in data["Landmarks"].items():
        new_x, new_y = transform_point(x, y)
        transformed["Landmarks"][key] = [new_x, new_y]
    return transformed

def process_dataset():
    files = sorted(f for f in os.listdir(INPUT_JSON_DIR) if f.endswith(".json"))
    for fname in tqdm(files):
        img_path = os.path.join(INPUT_IMG_DIR, fname.replace(".json", ".png"))
        json_path = os.path.join(INPUT_JSON_DIR, fname)
        out_img_path = os.path.join(OUTPUT_IMG_DIR, fname.replace(".json", ".png"))
        out_json_path = os.path.join(OUTPUT_JSON_DIR, fname)

        # Read image
        img = cv2.imread(img_path)
        if img is None:
            print(f"Skipping {img_path} (not found or invalid).")
            continue

        # Read JSON
        with open(json_path, 'r') as f:
            annotation = json.load(f)

        # Preprocess
        processed_img = preprocess_image(img)
        processed_annotation = transform_json(annotation)

        # Save image and updated annotation
        cv2.imwrite(out_img_path, processed_img)
        with open(out_json_path, 'w') as f:
            json.dump(processed_annotation, f, indent=2)

    print("All images and annotations have been preprocessed.")

process_dataset()

100%|██████████| 4000/4000 [00:16<00:00, 235.35it/s]

All images and annotations have been preprocessed.
